In [5]:
import time
import torch
from torch.distributions import Categorical, kl
import numpy as np
from tqdm import tqdm
# from d2l.torch import Animator

from net_deepaco import Net as Net_DeepACO
from net import Net
from aco import ACO
from faco import FACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f'Using device: {device}')

Using device: cuda:0


In [6]:
from xml.parsers.expat import model


@torch.no_grad()
def infer_instance(model, demands, distances, n_ants, t_aco_diff, use_deepaco=False):
    if model:
        model.eval()
        pyg_data = gen_pyg_data(demands, distances, device)
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        aco = FACO(
            distances=distances,
            demand=demands,
            n_ants=n_ants,
            heuristic=heu_mat,
            device=device
        ) if not use_deepaco else ACO(
            distances=distances,
            demand=demands,
            n_ants=n_ants,
            heuristic=heu_mat,
            device=device
        )
    else:
        aco = ACO(
            distances=distances,
            demand=demands,
            n_ants=n_ants,
            device=device
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    bar = tqdm(dataset, dynamic_ncols=True)
    start = time.time()
    for demands, distances in bar:
        results = infer_instance(model, demands, distances, n_ants, t_aco_diff)
        sum_results += results
        bar.set_description(f'Avg costs: {(sum_results/(bar.n)).cpu().numpy()}')
    end = time.time()
    
    return sum_results / len(dataset), end-start

@torch.no_grad()
def test_deepaco(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    bar = tqdm(dataset, dynamic_ncols=True)
    start = time.time()
    for demands, distances in bar:
        results = infer_instance(model, demands, distances, n_ants, t_aco_diff, use_deepaco=True)
        sum_results += results
        bar.set_description(f'Avg costs: {(sum_results/(bar.n)).cpu().numpy()}')
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on TSP20

DeepACO

In [7]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

net = Net_DeepACO().to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/cvrp{n_node}.pt', map_location=device))
avg_aco_best, duration = test_deepaco(test_list, net, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i])) 
  

Avg costs: [5.1372905 4.8985357 4.8496675 4.8395467]: 100%|██████████| 100/100 [02:39<00:00,  1.59s/it]

total duration:  159.09455394744873
T=1, average obj. is 5.0859174728393555.
T=10, average obj. is 4.849550247192383.
T=50, average obj. is 4.801170349121094.
T=100, average obj. is 4.79115104675293.


NGFACO

In [8]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

net = Net_DeepACO().to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/cvrp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i])) 
  

Avg costs: [5.195157  4.6852236 4.6506686 4.6506476]: 100%|██████████| 100/100 [01:01<00:00,  1.63it/s]

total duration:  61.52629733085632
T=1, average obj. is 5.143205642700195.
T=10, average obj. is 4.638371467590332.
T=50, average obj. is 4.604161739349365.
T=100, average obj. is 4.6041412353515625.


ACO

In [9]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)


avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

Avg costs: [7.92956  6.896563 6.603587 6.592754]:  67%|██████▋   | 67/100 [02:12<01:05,  1.98s/it]       


KeyboardInterrupt: 